# <center> 数据建模作业9：神经网络参数优化性能对比 </center>
<center>姓名：李先洋 学号：M201970007</center>

## 1.	描述性统计与分析
读入数据，对所有变量进行描述性统计。

每一个变量的均值、方差差异都很明显。尤其是x3 变量，均值方差与其他变量有较大的差别，所以先对所有变量进行归一化。

In [1]:
from sklearn.datasets import load_boston
import pandas as pd
from torch import nn
import torch
import numpy as np

df = load_boston()
data = np.concatenate([df.data,df.target.reshape(-1,1)],axis=1).astype(np.float32)
feature_names=list(df.feature_names)
feature_names.append('TARGET')
data_df = pd.DataFrame(data, columns=feature_names)
data_df.describe()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136780,0.069170,0.554695,6.284634,68.574898,3.795043,9.549407,408.237152,18.455534,356.674042,12.653064,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537109,2.164946,91.294861,7.141061,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377487,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440002,11.360000,21.200001
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.074999,5.188425,24.000000,666.000000,20.200001,396.225006,16.954999,25.000000
max,88.976196,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.899994,37.970001,50.000000


In [2]:
CHAS=data_df['CHAS']
TARGET=data_df['TARGET']
data_df = (data_df-data_df.mean())/(data_df.std())#做归一化处理
data_df['CHAS']=CHAS
# data_df['TARGET']=TARGET
data_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
count,5.060000e+02,5.060000e+02,506.000000,506.000000,506.000000,506.000000,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,506.000000,506.000000,5.060000e+02,5.060000e+02
mean,6.031142e-08,-1.507786e-08,-0.000003,0.069170,-0.000004,-0.000002,-6.935813e-07,-1.507786e-08,3.015571e-08,3.015571e-08,-0.000023,-0.000006,-6.785034e-08,-7.538928e-09
std,9.999999e-01,1.000000e+00,1.000000,0.253994,1.000000,1.000000,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000e+00,9.999999e-01
min,-4.193669e-01,-4.872402e-01,-1.556304,0.000000,-1.464437,-3.876415,-2.333129e+00,-1.265816e+00,-9.818712e-01,-1.312691e+00,-2.704725,-3.903336,-1.529613e+00,-1.906340e+00
25%,-4.105632e-01,-4.872402e-01,-0.866835,0.000000,-0.912130,-0.568069,-8.366207e-01,-8.048913e-01,-6.373311e-01,-7.668172e-01,-0.487580,0.204863,-7.986297e-01,-5.988631e-01
50%,-3.902802e-01,-4.872402e-01,-0.210893,0.000000,-0.144079,-0.108360,3.170672e-01,-2.790473e-01,-5.224844e-01,-4.642132e-01,0.274564,0.380804,-1.810744e-01,-1.449158e-01
75%,7.389288e-03,4.872403e-02,1.014992,0.000000,0.598083,0.482289,9.059009e-01,6.617161e-01,1.659603e+00,1.529413e+00,0.805756,0.433217,6.024225e-01,2.682576e-01
max,9.924109e+00,3.800474e+00,2.420167,1.000000,2.729641,3.551528,1.116389e+00,3.956602e+00,1.659603e+00,1.796417e+00,1.637185,0.440610,3.545262e+00,2.986505e+00


In [3]:
data=np.array(data_df)
np.random.shuffle(data)
train_data = data[:int(data.shape[0] * 0.8)]
# print(train_data.dtype)
test_data = data[int(data.shape[0] * 0.8):]
print(f'train_data.shape: {train_data.shape}, test_data.shape: {test_data.shape}')

train_data.shape: (404, 14), test_data.shape: (102, 14)


In [4]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
mlp= MLPRegressor(activation='logistic',max_iter=1000,verbose=True,solver='sgd',momentum=0)
print(mlp)


MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=1000, momentum=0,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='sgd', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)


In [5]:
mlp.fit(train_data[:,:-1],train_data[:,-1])
mlp_y_predict=mlp.predict(train_data[:,:-1])
train_loss = mean_squared_error(train_data[:,-1], mlp_y_predict)
mlp_y_predict=mlp.predict(test_data[:,:-1])
test_loss = mean_squared_error(test_data[:,-1], mlp_y_predict)
print(f'the mse on the train data :{train_loss}, on the test data: {test_loss}')

Iteration 1, loss = 0.61769271
Iteration 2, loss = 0.61290049
Iteration 3, loss = 0.60732314
Iteration 4, loss = 0.60091169
Iteration 5, loss = 0.59616125
Iteration 6, loss = 0.59525734
Iteration 7, loss = 0.59239114
Iteration 8, loss = 0.58875276
Iteration 9, loss = 0.58652752
Iteration 10, loss = 0.58276149
Iteration 11, loss = 0.58212464
Iteration 12, loss = 0.58248469
Iteration 13, loss = 0.58162897
Iteration 14, loss = 0.58352983
Iteration 15, loss = 0.58327017
Iteration 16, loss = 0.58376639
Iteration 17, loss = 0.58210111
Iteration 18, loss = 0.58018327
Iteration 19, loss = 0.57892791
Iteration 20, loss = 0.57641405
Iteration 21, loss = 0.57800850
Iteration 22, loss = 0.57519405
Iteration 23, loss = 0.57171155
Iteration 24, loss = 0.57095547
Iteration 25, loss = 0.57222726
Iteration 26, loss = 0.57257268
Iteration 27, loss = 0.56943430
Iteration 28, loss = 0.56927784
Iteration 29, loss = 0.56930644
Iteration 30, loss = 0.56550932
Iteration 31, loss = 0.56329517
Iteration 32, los

Iteration 295, loss = 0.39282469
Iteration 296, loss = 0.39199339
Iteration 297, loss = 0.39147686
Iteration 298, loss = 0.39105274
Iteration 299, loss = 0.39044298
Iteration 300, loss = 0.39023358
Iteration 301, loss = 0.38969026
Iteration 302, loss = 0.38956737
Iteration 303, loss = 0.38902479
Iteration 304, loss = 0.38862364
Iteration 305, loss = 0.38770666
Iteration 306, loss = 0.38744444
Iteration 307, loss = 0.38725775
Iteration 308, loss = 0.38638165
Iteration 309, loss = 0.38600984
Iteration 310, loss = 0.38572398
Iteration 311, loss = 0.38520415
Iteration 312, loss = 0.38482632
Iteration 313, loss = 0.38442429
Iteration 314, loss = 0.38417831
Iteration 315, loss = 0.38386873
Iteration 316, loss = 0.38356280
Iteration 317, loss = 0.38343349
Iteration 318, loss = 0.38298688
Iteration 319, loss = 0.38204338
Iteration 320, loss = 0.38153458
Iteration 321, loss = 0.38134549
Iteration 322, loss = 0.38088620
Iteration 323, loss = 0.38029913
Iteration 324, loss = 0.37991371
Iteration 

Iteration 545, loss = 0.30617099
Iteration 546, loss = 0.30589564
Iteration 547, loss = 0.30582898
Iteration 548, loss = 0.30598209
Iteration 549, loss = 0.30566978
Iteration 550, loss = 0.30541280
Iteration 551, loss = 0.30494864
Iteration 552, loss = 0.30399800
Iteration 553, loss = 0.30389313
Iteration 554, loss = 0.30347856
Iteration 555, loss = 0.30336145
Iteration 556, loss = 0.30295255
Iteration 557, loss = 0.30271481
Iteration 558, loss = 0.30249911
Iteration 559, loss = 0.30232949
Iteration 560, loss = 0.30212088
Iteration 561, loss = 0.30196312
Iteration 562, loss = 0.30190037
Iteration 563, loss = 0.30176231
Iteration 564, loss = 0.30151799
Iteration 565, loss = 0.30132755
Iteration 566, loss = 0.30112447
Iteration 567, loss = 0.30100812
Iteration 568, loss = 0.30098371
Iteration 569, loss = 0.30092675
Iteration 570, loss = 0.30079854
Iteration 571, loss = 0.30111971
Iteration 572, loss = 0.30114159
Iteration 573, loss = 0.30096546
Iteration 574, loss = 0.30056081
Iteration 

Iteration 823, loss = 0.25255902
Iteration 824, loss = 0.25209517
Iteration 825, loss = 0.25207679
Iteration 826, loss = 0.25199233
Iteration 827, loss = 0.25179334
Iteration 828, loss = 0.25158710
Iteration 829, loss = 0.25130433
Iteration 830, loss = 0.25119142
Iteration 831, loss = 0.25109899
Iteration 832, loss = 0.25101805
Iteration 833, loss = 0.25089877
Iteration 834, loss = 0.25081340
Iteration 835, loss = 0.25068783
Iteration 836, loss = 0.25054455
Iteration 837, loss = 0.25040363
Iteration 838, loss = 0.25049925
Iteration 839, loss = 0.25044681
Iteration 840, loss = 0.25055873
Iteration 841, loss = 0.25062451
Iteration 842, loss = 0.25009142
Iteration 843, loss = 0.24991280
Iteration 844, loss = 0.24959973
Iteration 845, loss = 0.24946521
Iteration 846, loss = 0.24927622
Iteration 847, loss = 0.24923147
Iteration 848, loss = 0.24908901
Iteration 849, loss = 0.24893104
Iteration 850, loss = 0.24879230
Iteration 851, loss = 0.24871094
Iteration 852, loss = 0.24863355
Iteration 

c:\users\li\miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [6]:
mlp= MLPRegressor(activation='logistic',max_iter=1000,verbose=True,solver='sgd',momentum=0.9)
print(mlp)
mlp.fit(train_data[:,:-1],train_data[:,-1])
mlp_y_predict=mlp.predict(train_data[:,:-1])
train_loss = mean_squared_error(train_data[:,-1], mlp_y_predict)
mlp_y_predict=mlp.predict(test_data[:,:-1])
test_loss = mean_squared_error(test_data[:,-1], mlp_y_predict)
print(f'the mse on the train data :{train_loss}, on the test data: {test_loss}')

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=1000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)
Iteration 1, loss = 0.56614273
Iteration 2, loss = 0.56303389
Iteration 3, loss = 0.55009215
Iteration 4, loss = 0.53703849
Iteration 5, loss = 0.53127022
Iteration 6, loss = 0.52791555
Iteration 7, loss = 0.52618313
Iteration 8, loss = 0.51687980
Iteration 9, loss = 0.50758978
Iteration 10, loss = 0.50121348
Iteration 11, loss = 0.49686643
Iteration 12, loss = 0.49076087
Iteration 13, loss = 0.48623695
Iteration 14, loss = 0.48487938
Iteration 15, loss = 0.48148955
Iteratio

Iteration 237, loss = 0.16906570
Iteration 238, loss = 0.16878892
Iteration 239, loss = 0.16828660
Iteration 240, loss = 0.16736677
Iteration 241, loss = 0.16804912
Iteration 242, loss = 0.16835826
Iteration 243, loss = 0.16687652
Iteration 244, loss = 0.16636918
Iteration 245, loss = 0.16603624
Iteration 246, loss = 0.16569974
Iteration 247, loss = 0.16544078
Iteration 248, loss = 0.16622211
Iteration 249, loss = 0.16571426
Iteration 250, loss = 0.16535936
Iteration 251, loss = 0.16471359
Iteration 252, loss = 0.16412808
Iteration 253, loss = 0.16405670
Iteration 254, loss = 0.16365037
Iteration 255, loss = 0.16330622
Iteration 256, loss = 0.16271340
Iteration 257, loss = 0.16265447
Iteration 258, loss = 0.16303241
Iteration 259, loss = 0.16343274
Iteration 260, loss = 0.16245929
Iteration 261, loss = 0.16167557
Iteration 262, loss = 0.16149726
Iteration 263, loss = 0.16221412
Iteration 264, loss = 0.16183063
Iteration 265, loss = 0.16150007
Iteration 266, loss = 0.16162669
Iteration 